In [5]:
pip install mlflow


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [17]:
df = pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [18]:
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]

In [19]:
# 🔍 Identifier les colonnes catégorielles (object ou string)
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# 🧱 Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", sparse_output=False), categorical_cols)
    ],
    remainder="passthrough"  # garde les colonnes numériques
)

# 🔁 Pipeline complet avec le modèle
model_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(C=1.0, max_iter=1000))
])

In [20]:
# 📚 Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Lancement du run MLflow
with mlflow.start_run():

    # 🔨 Entraînement du pipeline
    model_pipeline.fit(X_train, y_train)

    # 🔍 Prédictions
    y_pred = model_pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    # 📝 Log dans MLflow
    mlflow.log_param("C", 1.0)
    mlflow.log_param("max_iter", 1000)
    mlflow.log_metric("accuracy", acc)

    # 💾 Log du modèle complet (avec pipeline)
    mlflow.sklearn.log_model(model_pipeline, "model")

    # (Optionnel) sauvegarde locale du modèle
    joblib.dump(model_pipeline, "logistic_pipeline.pkl")

    print(f"✅ Accuracy: {acc:.4f} — Modèle loggué avec MLflow.")

2025/05/13 14:50:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Accuracy: 0.8533 — Modèle loggué avec MLflow.
